In [ ]:
from bs4 import BeautifulSoup as bs
from slugify import slugify
import requests
import json
import os

In [74]:
if not os.path.exists("Quests"):
    os.makedirs("Quests")

def add_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

## Get all quests

In [3]:
URL = "https://www.dofuspourlesnoobs.com/classeacutees-par-succegraves.html"
base_URL = "https://www.dofuspourlesnoobs.com"
success_index_page = requests.get(URL)
success_index_soup = bs(success_index_page.content, "html.parser")

### Parse "Listing des quêtes par succès" page

In [12]:
success_info_list = {}

success_table = success_index_soup.find_all("table", class_="wsite-multicol-table")
success_table_list = [elem.find_all("td", class_="wsite-multicol-col") for elem in success_table]

#### success_table_list :
- [0] I- Quêtes par succès
- [1] II- Quêtes événementielles par succès (part 1)
- [2] II- Quêtes événementielles par succès (part 2)
- [3] III- Quêtes non disponibles dans les succès
- [4] Anciennes quêtes de Dofus
- [5+] Nothing

In [70]:
def get_success_list(bs_table_list):
    list = {}
    succes_group = ""
    for bs_table_group in bs_table_list:
        for bs_table in bs_table_group:
            for div in bs_table.find_all("div", class_="paragraph"):
                if div.find("a") and succes_group != "":
                    for link in div.find_all("a"):
                        list[succes_group][link.text.strip()] = {"link": link["href"], "quests" :{}}
                else:
                    succes_group = div.text.encode('utf-8', 'ignore').decode('utf-8').replace('\ufeff', '').replace('\ufefb', '').strip()
                    list[succes_group] = {}
    return list

def get_direct_quests(bs_table_list):
    list = {}
    for bs_table_group in bs_table_list:
        for bs_table in bs_table_group:
            for link in bs_table.find_all("a"):
                list[link.text.strip()] = link["href"]

In [83]:
success_index_list = get_success_list([success_table_list[2]])
nosuccess_quests = get_success_list([success_table_list[4]])

In [84]:
success_index_list

{'Halouine :': {'Halouine.': {'link': '/halouine.html', 'quests': {}},
  'Même pas peur.': {'link': '/meme-pas-peur.html', 'quests': {}}},
 'Île de Pwâk\xa0:': {'Île de Pwâk.': {'link': '/ile-de-pwak.html',
   'quests': {}},
  'Un brin de mystère.': {'link': '/un-brin-de-mystere.html', 'quests': {}},
  'Promenades digestives.': {'link': '/promenades-digestives.html',
   'quests': {}}}}

### Get all quests for each success

In [88]:
def get_quests_from_succes_page(bs_page):
    quests = {}
    for link in bs_page.find_all("a"):
        try:
            quests[link.text.strip()] = {"link": link["href"]}
        except Exception as x:
            print("ERROR :", x)
            quests["[ERROR]"+link.text.strip()] = {"link": None, "error": x}
    return quests

def add_quests_to_success(succes_dict):
    category = ""
    for category, success in succes_dict.items():
        add_folder("Quests/" + slugify(category))
        for success_key, success_elem in success.items():
            page_url = f"{base_URL}{success_elem["link"]}"
            page_soup = bs(requests.get(page_url).content).find("div", id="wsite-content")
            success_elem['quests'].update(get_quests_from_succes_page(page_soup))
    
    return succes_dict

In [ ]:
add_quests_to_success(success_index_list)
success_index_list


ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'
ERROR : 'href'


{'Halouine :': {'Halouine.': {'link': '/halouine.html',
   'quests': {'': {'link': '/uploads/1/3/0/1/13010384/20_20_orig.png'},
    "I- Les quêtes d'Halouine.": {'link': '#quetes'},
    "II- Les donjons d'Halouine.": {'link': '#donjons'},
    "III- Les objets d'apparat et leur obtention.": {'link': '#apparats'},
    "IV- L'attitude Halouine.": {'link': '#attitude'},
    "V- Les friandises d'Halouine.": {'link': '#friandises'},
    "VI- La malédiction d'Halouine.": {'link': '#malediction'},
    'VII- Ornements, titres, émoticônes, havre-sac.': {'link': '#ornements'},
    '[ERROR]': {'link': None, 'error': KeyError('href')},
    'Même pas peur': {'link': '/meme-pas-peur.html'},
    'Des bonbons ou un sort.': {'link': '/des-bonbons-ou-un-sort.html'},
    'Une Citwouille pour Halouine.': {'link': '/une-citwouille-pour-halouine.html'},
    'Le rituel de la bière.': {'link': '/le-rituel-de-la-biere.html'},
    'Une friandise pas comme les autres.': {'link': '/une-friandise-pas-comme-les-autr

: 

## Download all pages

In [42]:
for key in dungeon_info_list.keys():
    dungeon = dungeon_info_list[key]
    page_url = f"{base_URL}{dungeon["link"]}"
    page_soup = bs(requests.get(page_url).content).find("div", id="wsite-content")
    for div in page_soup:
        string = str(div)
        if "PUBLICITE" in string or "data-ad-text" in string:
            div.decompose()
    with open("Dungeons" + dungeon["link"], "w", encoding="utf-8")as file:
        file.write(str(page_soup))
    with open("Dungeons/txt" + dungeon["link"], "w", encoding="utf-8")as file:
        file.write(page_soup.get_text())